In [1]:
from solveLP import optimisePortfolio
import pandas as pd
import numpy as np

In [2]:
M = 90 # dimension of returns
K = 2 # number of regimes
L = {1: 1, 2: 1} # number of clusters in each regime

conf = 0.85 # confidence level for cVaR
theta = {(1,1): 0.02, (2,1): 0.07} # ambiguity level for each cluster in each regime; (k,l)
p = {(1,1): 1, (2,1): 1} # probability of each cluster

targ = 0.03 #target return

In [3]:
returns = pd.read_csv('returns.csv')
factors = pd.read_csv('factors.csv')
returns['regimeName'] = factors['regimeName']

period = 36
weights = []

for i in range(120-period):
    aux = returns[i:i+period]
    
    # find out the probability of the regime for the next period
    if aux['regimeName'].iloc[-1] == 'Low Volatility':
        w = {1: 0.8, 2: 0.2}
    
    else:
        w = {1: 0.3, 2: 0.7}
    
    
    df = {1: aux[aux['regimeName']=='Low Volatility'].drop(columns=['regimeName']),
          2: aux[aux['regimeName']=='High Volatility'].drop(columns=['regimeName'])}
    
    df[1].index = range(1,len(df[1])+1)
    df[2].index = range(1,len(df[2])+1)
    
    N = {(1,1): len(df[1]), (2,1): len(df[2])}
    r = {}
    
    for k in range(1, K+1):
        for l in range(1, L[k]+1):
            for n in range(1, N[(k,l)]+1):
                for i in range(1,M+1):
                    r[(k,l,n,i)] = df[k].loc[n][i-1]
    
    weights.append(optimisePortfolio(M, K, L, N, conf, w, theta, p, targ, r))

In [4]:
ret = returns.drop(columns=['regimeName'])
drReturn = np.concatenate((np.zeros(period), np.array([np.dot(weights[i][0], ret.iloc[period+i]) for i in range(len(weights))])))
drReturn

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.00688856,  0.04256485,  0.03770536, -0.24625964,
        0.02578337,  0.17819018,  0.040348  ,  0.04511351,  0.09073702,
       -0.10664528, -0.01249146,  0.01375095,  0.02261599,  0.08618622,
       -0.02866428,  0.22545624, -0.07086038,  0.12499468, -0.01546059,
       -0.03105555, -0.09843641,  0.01415516, -0.09633401,  0.02860281,
        0.11181196,  0.03933397, -0.01783486, -0.02661183,  0.08565625,
        0.20671971, -0.01475544, -0.02514691,  0.06813537,  0.09

In [5]:
factors['drReturn'] = drReturn

In [6]:
factors

,period,regime,regimeName,mrp,smb,hml,drReturn
0,1,0,Low Volatility,0.036376,0.183305,0.009808,0.000000
1,2,1,High Volatility,-0.007027,0.111461,-0.066324,0.000000
2,3,1,High Volatility,-0.020925,0.257004,-0.017614,0.000000
3,4,0,Low Volatility,0.165771,-0.055391,-0.010676,0.000000
4,5,0,Low Volatility,-0.024866,0.044256,0.034019,0.000000
5,6,0,Low Volatility,0.047414,0.067211,-0.020243,0.000000
6,7,0,Low Volatility,0.071379,-0.080831,-0.055200,0.000000
7,8,0,Low Volatility,-0.010067,-0.025086,0.065695,0.000000
8,9,0,Low Volatility,0.270712,0.038581,0.017654,0.000000
9,10,0,Low Volatility,0.247991,0.089310,0.162488,0.000000


In [7]:
theta

{(1, 1): 0.02, (2, 1): 0.07}

In [8]:
factors.to_csv('portfolioReturns.csv', index=False)